### Forecasting Algorithm reusable code


#### Loading Library

In [201]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from statsmodels.tsa.statespace.sarimax import SARIMAX
import xgboost as xgb
from lightgbm import LGBMRegressor
from statsmodels.tsa.holtwinters import ExponentialSmoothing, SimpleExpSmoothing, Holt
from sklearn.model_selection import GridSearchCV

#### Defining Functions to create holiday and discount flag in the input dataset

In [137]:
def create_features(df, rolling_windows, lag_periods, holiday_data, discount_data):
    # Create additional features
    df['is_holiday'] = df['date'].apply(lambda x: 1 if is_holiday(x, holiday_data) else 0)
    df['discount_flag'] = df['date'].apply(lambda x: get_discount_flag(x, discount_data))

    # Create rolling mean features
    for window in rolling_windows:
        df[f'rolling_mean_{window}'] = df['sales'].rolling(window).mean()

    # Create lag features
    for lag in lag_periods:
        df[f'lag_{lag}'] = df['sales'].shift(lag)

    # Remove rows with NaN values
    df.dropna(inplace=True)

    return df


def is_holiday(date, holiday_data):
    # Function to determine if a given date is a holiday
    # Use the holiday_data DataFrame to check if the date is a holiday
    holiday_flag = holiday_data[holiday_data['date'] == date]['flag'].values
    if len(holiday_flag) > 0:
        return bool(holiday_flag[0])
    else:
        return False


def get_discount_flag(date, discount_data):
    # Function to determine the discount flag for a given date
    # Use the discount_data DataFrame to check if the date has a discount flag
    discount_flag = discount_data[discount_data['date'] == date]['discount_flag'].values
    if len(discount_flag) > 0:
        return discount_flag[0]
    else:
        return 0


#### Reading Input data

In [157]:
# Load the dataset
data = pd.read_csv('store_sales.csv')  # Replace with the actual filename and path of your dataset
holiday_data = pd.read_csv('holiday_data.csv')
discount_data = pd.read_csv('discount_data.csv')  # Replace with the actual filename and path of your discount data

#### Operations

In [158]:
# Convert 'date' column to datetime type
data['date'] = pd.to_datetime(data['date'], format='%d-%m-%Y', dayfirst=True)

# Specify the rolling mean windows and lag periods
rolling_windows = [2, 3, 6, 9, 12, 18]
lag_periods = [2, 3, 4, 5, 6, 7, 8]

# Specify the number of weeks for training and testing
num_train_weeks = len(data['date'].unique()) - 8  # Train on all weeks except the last 8
num_test_weeks = 8  # Test on the last 8 weeks

predictions_list = []

#### Creating the Store dataframe with all the features

In [196]:
store_df=pd.DataFrame()

for store in data['store'].unique():
    store_data = data[data['store'] == store].copy()

    # Sort the data by date
    store_data.sort_values('date', inplace=True)

    # inserting the missing dates
    min_date = store_data['date'].min()
    max_date = store_data['date'].max()
    date_range = pd.date_range(min_date, max_date, freq='W-FRI')
    missing_dates = set(date_range) - set(store_data['date'])
    missing_data = pd.DataFrame({'date': list(missing_dates)})
    store_data = pd.concat([store_data, missing_data]).sort_values('date')

    # Fill missing sales values with 0
    store_data['sales'].fillna(0, inplace=True)

    # Create additional features
    store_data = create_features(store_data, rolling_windows, lag_periods, holiday_data, discount_data)
    store_df = pd.concat([store_df, store_data])
    
#store_df.set_index('date', inplace=True)

In [197]:
store_df.head(2)

,date,store,sales,is_holiday,discount_flag,rolling_mean_2,rolling_mean_3,rolling_mean_6,rolling_mean_9,rolling_mean_12,rolling_mean_18,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8
17,2010-06-04,100.0,50188543.12,0,0,48973022.84,4.768872e+07,4.676743e+07,4.643200e+07,4.645290e+07,4.669804e+07,45120108.06,45330080.20,48503243.52,43705126.71,44734452.56,45183667.08,47365290.44
18,2010-06-11,100.0,47826546.72,0,0,49007544.92,4.859086e+07,4.745434e+07,4.648325e+07,4.668936e+07,4.659114e+07,47757502.56,45120108.06,45330080.20,48503243.52,43705126.71,44734452.56,45183667.08


In [1]:
pip install fbprophet

^C
Note: you may need to restart the kernel to use updated packages.


#### Model Training & Prediction 

In [209]:
import pandas as pd
from prophet import Prophet
from sklearn.model_selection import TimeSeriesSplit

# Prepare the data for Prophet
df = store_df[['date', 'sales']]
df.columns = ['ds', 'y']

# Convert the 'ds' column to a pandas datetime object
df['ds'] = pd.to_datetime(df['ds'])

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'changepoint_prior_scale': [0.01, 0.1, 1],
    'seasonality_prior_scale': [0.01, 0.1, 1],
    'seasonality_mode': ['additive', 'multiplicative']
}

# Initialize variables to store the best model and hyperparameters
best_model = None
best_params = None
best_score = float('-inf')

# Perform hyperparameter tuning and cross-validation
tscv = TimeSeriesSplit(n_splits=3)
for train_index, val_index in tscv.split(df):
    train_data = df.iloc[train_index]
    val_data = df.iloc[val_index]

    for params in param_grid.values():
        for param_value in params:
            # Create a Prophet model with the current hyperparameters
            model = Prophet(**{param_name: param_value for param_name, _ in param_grid.items()})

            # Fit the model to the training data
            model.fit(train_data)

            # Make predictions on the validation data
            forecast = model.predict(val_data[['ds']])

            # Calculate the evaluation metric (e.g., mean squared error)
            # You should replace the metric with your desired evaluation metric
            # Here, we calculate the mean squared error
            mse = ((forecast['yhat'] - val_data['y']) ** 2).mean()

            # Check if the current model has a better score
            if mse > best_score:
                best_model = model
                best_params = {param_name: param_value for param_name, param_value in zip(param_grid.keys(), params)}
                best_score = mse

# Create future dates for forecasting
num_weeks = 8  # Change this value as per your requirement
future_dates = best_model.make_future_dataframe(periods=num_weeks, freq='W-FRI')

# Make predictions for the future dates
forecast = best_model.predict(future_dates)

# Extract the relevant columns from the forecast
forecast_df = forecast[['ds', 'yhat']]
forecast_df.columns = ['forecast_weeks', 'forecast_sales']

# Display the forecasted weeks and sales
print(forecast_df.tail(num_weeks))

# Display the best hyperparameters and score
print("Best Hyperparameters:", best_params)
print("Best Score:", best_score)


C:\Users\anjis\AppData\Local\Temp\ipykernel_11952\357916059.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ds'] = pd.to_datetime(df['ds'])


ValueError: seasonality_mode must be "additive" or "multiplicative"

In [ ]:
# Iterate over each store
skipped_store=[]
for store in data['store'].unique():
    store_data = data[data['store'] == store].copy()

    # Sort the data by date
    store_data.sort_values('date', inplace=True)

    # inserting the missing dates
    min_date = store_data['date'].min()
    max_date = store_data['date'].max()
    date_range = pd.date_range(min_date, max_date, freq='W-FRI')
    missing_dates = set(date_range) - set(store_data['date'])
    missing_data = pd.DataFrame({'date': list(missing_dates)})
    store_data = pd.concat([store_data, missing_data]).sort_values('date')

    # Fill missing sales values with 0
    store_data['sales'].fillna(0, inplace=True)

    # Create additional features
    store_data = create_features(store_data, rolling_windows, lag_periods, holiday_data, discount_data)

    # Split the data into features (X) and target variable (y)
    X = store_data.drop(['store', 'date', 'sales'], axis=1)
    y = store_data['sales']
    X_train = X.iloc[:num_train_weeks]
    y_train = y.iloc[:num_train_weeks]
    
    if len(X_train) < 2 or len(y_train) < 2:
        skipped_store.append(store)
        continue

    # Create the scaler for feature scaling
    scaler = StandardScaler()

    # Define the models for time series and regression
    models = [
        #('time_series_1', SimpleExpSmoothing(endog=y_train)),
        ('time_series_2', Prophet()),
        ('time_series_3', SARIMAX(endog=y_train)),
        ('regression_1', LinearRegression()),
        ('regression_2', DecisionTreeRegressor()),
        ('regression_3', SVR()),
        ('regression_4', KNeighborsRegressor()),
        ('regression_5', RandomForestRegressor()),
        ('regression_6', MLPRegressor()),
        ('regression_7', GaussianProcessRegressor()),
        ('regression_8', xgb.XGBRegressor()),
        ('regression_9', RandomForestRegressor()),
        ('regression_10', LGBMRegressor())
    ]

    # Define the parameter grids for each model
    param_grids = {
        'time_series_1': {
            
        },
        'time_series_2': {
            
        },
        'time_series_3': {
            'order': [(1, 0, 0), (0, 0, 1)]
        },
        'regression_1': {
            'fit_intercept': [True, False],
            'normalize': [True, False]
        },
        'regression_2': {
            'max_depth': [None, 5, 10]
        },
        'regression_3': {
            'C': [0.1, 1.0, 10.0],
            'epsilon': [0.1, 0.01, 0.001]
        },
        'regression_4': {
            'n_neighbors': [3, 5, 7]
        },
        'regression_5': {
            'n_estimators': [100, 200, 500],
            'max_features': ['auto', 'sqrt'],
            'max_depth': [None, 5, 10]
        },
        'regression_6': {
            'hidden_layer_sizes': [(100,), (50, 50), (20, 20, 20)],
            'activation': ['relu', 'tanh'],
            'solver': ['adam', 'lbfgs']
        },
        'regression_7': {
            'alpha': [0.1, 1.0, 10.0]
        },
        'regression_8': {
            'n_estimators': [100, 200, 500],
            'max_depth': [3, 6, 9]
        },
        'regression_9': {
            'n_estimators': [100, 200, 500],
            'max_features': ['auto', 'sqrt'],
            'max_depth': [None, 5, 10]
        },
        'regression_10': {
            'n_estimators': [100, 200, 500],
            'max_depth': [3, 6, 9]
        }
    }

    # Perform time series forecasting and regression for each model
    for model_name, model in models:
        
            if model_name == 'time_series_2':  # Prophet model
                print(f"Training {model_name} for store {store}...")

                # Create the pipeline for the Prophet model
                steps = [
                    ('scaler', scaler),
                    ('feature_selection', SelectKBest(score_func=f_regression)),
                    ('model', model)
                ]
                pipeline = Pipeline(steps)

                # Perform time series cross-validation with grid search
                tscv = TimeSeriesSplit(n_splits=5)
                grid_search = GridSearchCV(pipeline, param_grids[model_name], scoring='neg_mean_squared_error', cv=tscv)

                # Train the model on the training data
                grid_search.fit(X_train, y_train)

                # Get the best model
                best_model = grid_search.best_estimator_

                # Perform predictions for the test set
                X_test = X.iloc[num_train_weeks:num_train_weeks + num_test_weeks]
                predictions = best_model.predict(X_test)

                # Append the predictions to the list
                for i, prediction in enumerate(predictions):
                    predictions_list.append((store, model_name, prediction, X_test.index[i]))

                # Update the features and target variable for the next iteration
                X = X.append(X_test)
                y = y.append(pd.Series(predictions, index=X_test.index))
    
            else:

                print(f"Training {model_name} for store {store}...")

                # Create the pipeline for each model
                steps = [
                    ('scaler', scaler),
                    ('feature_selection', SelectKBest(score_func=f_regression)),
                    ('model', model)
                ]
                pipeline = Pipeline(steps)

                # Perform time series cross-validation with grid search
                tscv = TimeSeriesSplit(n_splits=5)
                grid_search = GridSearchCV(pipeline, param_grids[model_name], scoring='neg_mean_squared_error', cv=tscv)

                # Train the model on the training data

                grid_search.fit(X_train, y_train)

                # Get the best model
                best_model = grid_search.best_estimator_

                # Perform predictions for the test set
                X_test = X.iloc[num_train_weeks:num_train_weeks + num_test_weeks]
                predictions = best_model.predict(X_test)

                # Append the predictions to the list
                for i, prediction in enumerate(predictions):
                    predictions_list.append((store, model_name, prediction, X_test.index[i]))

                # Update the features and target variable for the next iteration
                X = X.append(X_test)
                y = y.append(pd.Series(predictions, index=X_test.index))


# Create a DataFrame for the predictions
predictions_df = pd.DataFrame(predictions_list, columns=['store', 'model', 'prediction', 'date'])

# Calculate the accuracy of the ensemble model on the test set
test_set = data[data['week'] > data['week'].max() - num_test_weeks]  # Filter the test set
test_set = test_set[['store', 'date', 'sales']]  # Select relevant columns
merged_df = pd.merge(predictions_df, test_set, on=['store', 'date'], how='inner')  # Merge predictions and test set
mse = mean_squared_error(merged_df['sales'], merged_df['prediction'])
accuracy = 1 - (mse / np.var(merged_df['sales']))

print("Ensemble Model Accuracy:", accuracy)

# Sort the merged dataframe by date
merged_df.sort_values('date', inplace=True)

# Calculate the inverse error for each model
merged_df['inverse_error'] = 1 / (merged_df['prediction'] - merged_df['sales']) ** 2

# Calculate the contribution of each model to the ensemble prediction
model_contributions = merged_df.groupby(['store', 'model'])['inverse_error'].sum()
model_contributions = model_contributions.groupby('store').apply(lambda x: x / x.sum())

# Sort the contributions in descending order and select the top 3 models
top_models = model_contributions.groupby('store').apply(lambda x: x.nlargest(3)).reset_index()

# Filter the predictions dataframe based on the top models
final_predictions = pd.merge(predictions_df, top_models, on=['store', 'model'], how='inner')

# Print the final dataframe with the ensemble prediction
print(final_predictions)